In [ ]:
import os
import sys
import functools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime

sys.path.append("../")

In [ ]:
# from pynhm import load_wbl_output, load_prms_input
import pynhm

In [ ]:
wbal_df = pynhm.load_wbl_output("../prms_models/box01")
wbal_df

In [ ]:
wbal_df.columns

# Load PRMS5 Input and Output Files for NHM Basin

In [ ]:
input_data_path = "../prms_models/box01/input"
output_data_path = "../prms_models/box01/output"

In [ ]:
input_data_path
datanames = ["precipitation", "temp_min", "temp_max"]
filenames = ["prcp.cbh", "tmin.cbh", "tmax.cbh"]
input_df = pynhm.load_prms_input(input_data_path, datanames, filenames)
input_df

In [ ]:
# load prms output
csvfiles = [f for f in os.listdir(output_data_path) if f.endswith(".csv")]
csvfiles.remove("stats.csv")
output_df = pynhm.load_prms_output(output_data_path, csvfiles)
output_df

In [ ]:
# load the prms5 stats.csv file
fname = os.path.join(output_data_path, "stats.csv")
prms_stats_df = pynhm.load_prms_statscsv(fname)
prms_stats_df.columns

In [ ]:
prms_stats_df.loc[:, ["date", "basin_intcp_stor", "basin_intcp_evap"]]

# Run Python NHM

In [ ]:
start_time = datetime.datetime(1980, 10, 1)
end_time = datetime.datetime(2019, 12, 31)
# end_time = datetime.datetime(1981, 10, 1)
delta_time = datetime.timedelta(days=1)
print(start_time, end_time)

In [ ]:
# for now assume rainfall is the precipitation input
#date_mask = (input_df["date"] >= start_time) & (input_df["date"] <= end_time)
#rainfall = input_df.loc[date_mask, ["precipitation"]]
#rainfall = np.array(rainfall).flatten()
#rainfall

# assume that rainfall is the hru_ppt data in the NHM output
date_mask = (output_df["date"] >= start_time) & (output_df["date"] <= end_time)
rainfall = output_df.loc[date_mask, ["hru_ppt"]]
rainfall = np.array(rainfall).flatten()
rainfall

In [ ]:
# for now provide potential ET as basin_potet prms output
date_mask = (prms_stats_df["date"] >= start_time) & (prms_stats_df["date"] <= end_time)
potential_evapotranspiration = prms_stats_df.loc[date_mask, ["basin_potet"]]
potential_evapotranspiration = np.array(potential_evapotranspiration).flatten()
potential_evapotranspiration.shape, potential_evapotranspiration

In [ ]:
prms_stats_df.columns

In [ ]:
wbal_df["soilzone_last_sm"].plot()

In [ ]:
plt.plot(wbal_df["soilzone_last_sm"], output_df["soil_moist_ante"], "bo")

In [ ]:
# for now provide previous soil moisture for runoff calculation
date_mask = (output_df["date"] >= start_time) & (output_df["date"] <= end_time)
soil_moisture0_in = output_df.loc[date_mask, ["soil_moist_ante"]].to_numpy().flatten()
soil_moisture0_in.shape, soil_moisture0_in

In [ ]:
# run a pynhm run
print("Initializing...")
#rainfall = [4.5, 0., 0., 0., 0.]
#potential_evapotranspiration = len(rainfall) * [1.]

# conversion factors
inch_to_meter = 0.0254
acre_to_meter_squared = 4046.8564224

# Assign input variables and convert to metric units
area = 1.0 * acre_to_meter_squared

# forcings
rainfall *= inch_to_meter
potential_evapotranspiration *= inch_to_meter

# interception
intcp_stor_max = 0.003847 * inch_to_meter
covden = 0.2308

# surface runoff
hru_percent_imperv = 0.0
imperv_stor_start = 0.0 * inch_to_meter
imperv_stor_max = 0.0500000 * inch_to_meter
carea_min = 0.0 # default
carea_max = 1.0
soil_rechr_max_frac = 0.8170124
smidx_coef = 0.0002278
smidx_exp = 1.2787037  # 1 / inch
smidx_exp = smidx_exp / inch_to_meter
soil_moisture0_in *= inch_to_meter

forcings = pynhm.AtmosphericForcings(
    rainfall, 
    potential_evapotranspiration
)

cnp = pynhm.prmsCanopy(
    id=0, 
    area=area, 
    forcing=forcings,
    verbose=False,
    intcp_stor_start=0., 
    intcp_stor_max=intcp_stor_max, 
    covden=covden
)

sro = pynhm.prmsSurfaceRunoff(
    id=1, 
    area=area, 
    forcing=forcings,
    verbose=False,
    hru_percent_imperv=hru_percent_imperv, 
    imperv_stor_start=imperv_stor_start,
    imperv_stor_max=imperv_stor_max,
    carea_min=carea_min,
    carea_max=carea_max,
    soil_rechr_max_frac=soil_rechr_max_frac,
    smidx_coef=smidx_coef,
    smidx_exp=smidx_exp,
    soil_moisture0_in=soil_moisture0_in,
)

# connect storage units
cnp.register_recipient(sro, "net_precipitation")

# create list of storage units in order of flow calculation
storage_units = [cnp, sro]

time_length = float(delta_time.days)
verbose = False
current_time = start_time
itime_step = 0

driver = pynhm.driver(current_time, end_time, delta_time, forcings, storage_units, verbose)

driver.run()

driver.finalize()


# Process Canopy Budget

In [ ]:
# output for python version
cnp_df = pd.DataFrame(np.array(cnp.output_data), columns=cnp.output_column_names)
cnp_df

In [ ]:
# calculate english equivalents for comparison with prms
conversion = [
    ("precip", 1. / area / inch_to_meter),
    ("aet", 1. / area / inch_to_meter),
    ("intcp_ds", 1. / area / inch_to_meter),
    ("net_precip", 1. / area / inch_to_meter),
    ("residual", 1. / area / inch_to_meter),
    ("intcp_stor_new", 1. / inch_to_meter),
    ("intcp_stor_old", 1. / inch_to_meter),
]
cnp_english_df = cnp_df.copy()
for name, conversion_factor in conversion:
    cnp_english_df[name] *= conversion_factor
cnp_english_df

In [ ]:
# PRMS output
output_df.loc[:, ["date", "hru_ppt", "net_ppt", "intcp_stor", "intcp_evap"]]

In [ ]:
d1 = np.array(-cnp_english_df["net_precip"])
d2 = np.array(output_df["net_ppt"])
ax = plt.subplot(1, 1, 1)
ax.plot(d1, d2, 'bo')
ax.set_xlabel("python net_precip")
ax.set_ylabel("prms net_ppt")
dmax = max(d1.max(), d2.max())
ax.plot([0, dmax], [0, dmax], 'k-')

In [ ]:
err = d2 - d1
print(err.min(), err.max(), err.mean())

In [ ]:
ax = plt.subplot(1, 1, 1)
ax.plot(err, 'b-')
ax.set_xlabel("day")
ax.set_ylabel("net_ppt error (prms - python)")

# Process Surface Runoff

In [ ]:
# output for python version
sro_df = pd.DataFrame(np.array(sro.output_data), columns=sro.output_column_names)
sro_df

In [ ]:
# convert to english units for prms comparison
conversion = [
    ("net_precipitation", 1. / area / inch_to_meter),
    ("impervious_runoff", 1. / area / inch_to_meter),
    ("impervious_ds", 1. / area / inch_to_meter),
    ("impervious_et", 1. / area / inch_to_meter),
    ("pervious_runoff", 1. / area / inch_to_meter),
    ("infiltration", 1. / area / inch_to_meter),
    ("residual", 1. / area / inch_to_meter),
    ("impervious_stor_new", 1. / inch_to_meter),
    ("impervious_stor_old", 1. / inch_to_meter),
]
sro_english_df = sro_df.copy()
for name, conversion_factor in conversion:
    sro_english_df[name] *= conversion_factor
sro_english_df

In [ ]:
# PRMS output
output_df.loc[:, ["date", "net_ppt", "hru_sroffi", "hru_impervevap", "hru_impervstor", 
                  "hru_sroffp"]]

In [ ]:
d1 = np.array(-sro_english_df["impervious_runoff"])
d2 = np.array(output_df["hru_sroffi"])
ax = plt.subplot(1, 1, 1)
ax.plot(d1, d2, 'bo')
ax.set_xlabel("python impervious_runoff")
ax.set_ylabel("prms hru_sroffi")
dmax = max(d1.max(), d2.max())
ax.plot([0, dmax], [0, dmax], 'k-')

In [ ]:
d1 = np.array(-sro_english_df["impervious_et"])
d2 = np.array(output_df["hru_impervevap"])
ax = plt.subplot(1, 1, 1)
ax.plot(d1, d2, 'bo')
ax.set_xlabel("python impervious_et")
ax.set_ylabel("prms hru_impervevap")
dmax = max(d1.max(), d2.max())
ax.plot([0, dmax], [0, dmax], 'k-')

In [ ]:
d1 = np.array(-sro_english_df["pervious_runoff"])
d2 = np.array(output_df["hru_sroffp"])
ax = plt.subplot(1, 1, 1)
ax.plot(d1, d2, 'bo')
ax.set_xlabel("python pervious_runoff")
ax.set_ylabel("prms hru_sroffp")
dmax = max(d1.max(), d2.max())
ax.plot([0, dmax], [0, dmax], 'k-')

In [ ]:
err = d2 - d1
print(err.min(), err.max(), err.mean())

In [ ]:
ax = plt.subplot(1, 1, 1)
ax.plot(err, 'b-')
ax.set_xlabel("day")
ax.set_ylabel("runoff error (prms - python)")

In [ ]:
ax = plt.subplot(1, 1, 1)
ax.plot(d1, 'b')
ax.plot(d2, 'k')
ax.set_xlabel("day")
ax.set_ylabel("pervious runoff")